# Test Training and Inference Lambda Function with Sample Data

##### This notebook showcases the usage of the training and inference Lambda Function by creating sample data and invoking the Lambda functions.

## 01) Imports & Setup

In [ ]:
import boto3
from sklearn import datasets
import json
import matplotlib.pyplot as plt
import requests

In [ ]:
# boto 3 Lambda client to invoke the Lambda functions programmatically
lambda_client = boto3.client('lambda')

In [ ]:
# Set Lambda function names

# inference Lambda function
inference_function_name = '<INFERENCE_NAME>' # replace with your Lambda function name
# training Lambda function
training_function_name = '<TRAINING_NAME>' # replace with your Lambda function name

## 02) Test Access to Lambda Function

First we test whether we have sufficient IAM permissions to invoke the two Lambda functions

In [ ]:
# invoke Lambda function in "dry-run" mode
response = lambda_client.invoke(
    FunctionName=inference_function_name,
    InvocationType='DryRun',
    LogType='Tail'
)

# check if status indicates successful access
if response.get('StatusCode') == 204:
    print('Access to inference Lambda function successful')
else:
    print('Access to inference Lambda function not successful. Please make sure your notebook\'s IAM role has sufficient permissions to invoke the Lambda function.')

In [ ]:
# invoke Lambda function in "dry-run" mode
response = lambda_client.invoke(
    FunctionName=training_function_name,
    InvocationType='DryRun',
    LogType='Tail'
)

# check if status indicates successful access
if response.get('StatusCode') == 204:
    print('Access to training Lambda function successful')
else:
    print('Access to training Lambda function not successful. Please make sure your notebook\'s IAM role has sufficient permissions to invoke the Lambda function.')

## 03) Test Lambda Function

### 03a) Training

Now that we have ensured sufficient IAM permissions, we can test the training Lambda functions with sample data.

In [ ]:
# create sample data
X, y = datasets.make_regression(
    n_samples=500,
    n_features=1,
    n_informative=1,
    n_targets=1,
    bias=20,
    noise=30
)

In [ ]:
# inspect the training data
plt.scatter(X, y,  color='black')

We can see that our data can be approximated with a linear regression.

In [ ]:
# prepare payload
data = {
    "data": {
        "X": X.tolist(),
        "y": y.tolist()
    }
}
event = {
    "body": json.dumps(data)
}
payload = json.dumps(event)

In [ ]:
# invoke training Lambda function
# the first call will take longer than succeeding ones, as the Lambda function warms up after a period of inactivates.
response = lambda_client.invoke(
    FunctionName=training_function_name,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=payload
)

In [ ]:
# investigate response
response_payload = response.get('Payload').read()
print(json.loads(response_payload))

### 03b) Inference

Now that we have successfully invoked the training Lambda function, which trained a regression model and saved the trained model to s3,
we can test the inference Lambda function. It will download the latest trained model from s3 and make predictions based on the
data we send in with the invocation call.

In [ ]:
# create sample inference data
X_test, y_test = datasets.make_regression(
    n_samples=10,
    n_features=1,
    n_informative=1,
    n_targets=1,
    bias=30,
    noise=50
)

In [ ]:
# prepare payload
data = {
    "data": X_test.tolist()
}
event = {
    "body": json.dumps(data)
}
payload = json.dumps(event)

In [ ]:
# invoke inference Lambda function
# the first call will take longer than succeeding ones, as the Lambda function warms up after a period of inactivates.
response = lambda_client.invoke(
    FunctionName=inference_function_name,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=payload
)
response_payload = response.get('Payload').read()

In [ ]:
# unpack response
response_body = json.loads(response_payload).get('body')
prediction = json.loads(response_body).get('prediction')
prediction = json.loads(prediction)

In [ ]:
# visualize prediction vs. ground truth
plt.scatter(X_test, y_test,  color='black')
plt.plot(X_test, prediction, color='blue', linewidth=3)
plt.show()

The above plot visualizes the ground truth (black dots) vs our model prediction (blue line)

## 04) Test with API Gateway

Instead of invoking the Lambda functions directly, we can also use the deployed API gateway, which could be used to offer the train and inference functionality to external applications. 

### 04a) Training

In [ ]:
# setup API parameters
url = '<URL>' #paste your API URL here
headers = {'Content-Type': "application/json", 'Accept': "application/json"}

# format payload
data = {
    "data": {
        "X": X.tolist(),
        "y": y.tolist()
    }
}

In [ ]:
# invoke API
response = requests.post(url, json=data, headers=headers)

In [ ]:
# investigate response
response_content = json.loads(response.content)
print(response_content)

### 04b) Inference

In [ ]:
# setup API parameters
url = '<URL>' #paste your API URL here
headers = {'Content-Type': "application/json", 'Accept': "application/json"}

# format payload
data = {
    "data": X_test.tolist()
}

In [ ]:
# invoke API
response = requests.post(url, json=data, headers=headers)

In [ ]:
# investigate response
response_content = json.loads(response.content)
print(response_content)